In [1]:
# 필요한 라이브러리들 (사용 가능한 라이브러리)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [2]:
# 1. LR
# 모델식과 W, b를 직접 정의하고, requires_grad=True로 놓았다.
# ** 기본적인 pytorch의 코드 작성 방법
# 1. x_train, y_train이 있음
# 2. x_train을 선형 조합 OR 기타 등등 식으로 계산하여 prediction을 만드는 식에서,
# ~2. 그 식의 파라미터를 설정(LR의 경우, W와 b를 정의하면 됨)
# 3. optimizer를 설정
# 4. prediction을 이용해 cost function 정의하고,
# ~4. step을 밟아가며, optimizer.zero_grad(), cost.backward(), optimizer.step() 을 수행

In [3]:
# 1. 데이터 준비 : (3, 1). 데이터 하나는 (1,)의 사이즈를 가짐. 
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

In [4]:
# 2. W, b 설정
W = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [5]:
# 3. optimizer 설정
optimizer = optim.SGD([W, b], lr=0.01)

In [6]:
# 4. 반복문에서 prediction 및 cost를 계산하고, optimizer를 업데이트
nb_epochs = 2000
for epoch in range(nb_epochs + 1):
    prediction = x_train * W + b
    cost = torch.mean((prediction-y_train)**2)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch%200 == 0:
        print(f"Epoch {epoch}, Cost {cost:.4f}, W, b {W.item()}, {b.item()}")

Epoch 0, Cost 18.6667, W, b 0.18666668236255646, 0.07999999821186066
Epoch 200, Cost 0.0298, W, b 1.80009925365448, 0.4544214904308319
Epoch 400, Cost 0.0114, W, b 1.876473307609558, 0.28080499172210693
Epoch 600, Cost 0.0043, W, b 1.9236682653427124, 0.1735200434923172
Epoch 800, Cost 0.0017, W, b 1.952831506729126, 0.10722480714321136
Epoch 1000, Cost 0.0006, W, b 1.9708527326583862, 0.06625857204198837
Epoch 1200, Cost 0.0002, W, b 1.9819889068603516, 0.04094360023736954
Epoch 1400, Cost 0.0001, W, b 1.9888702630996704, 0.025300635024905205
Epoch 1600, Cost 0.0000, W, b 1.9931223392486572, 0.015634294599294662
Epoch 1800, Cost 0.0000, W, b 1.995750069618225, 0.009661080315709114
Epoch 2000, Cost 0.0000, W, b 1.9973737001419067, 0.005970162805169821


In [7]:
# 주의 !!!!!!!!!!!! 
# x_train.shape : (3, 1)
# W.shape : (1,)
# x_train.matmul(W) 하면, 왜 (3,) 모양이 되어서 나오는 걸까
x_train @ W

tensor([1.9974, 3.9947, 5.9921], grad_fn=<MvBackward0>)

In [8]:
x_train * W

tensor([[1.9974],
        [3.9947],
        [5.9921]], grad_fn=<MulBackward0>)

In [9]:
# 중간 과정의 shape를 항상 잘 관찰해야 할 듯

In [10]:
# 2. MLR
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  80], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 100
for epoch in range(nb_epochs+1):
    prediction = x_train @ W + b
    cost = torch.mean((prediction-y_train)**2)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if epoch%20==0:
        print(f"Epoch : {epoch}, Cost : {cost}, W : {W.squeeze().detach()}, b : {b.item():.4f}")

Epoch : 0, Cost : 29661.80078125, W : tensor([0.2940, 0.2936, 0.2902]), b : 0.0034
Epoch : 20, Cost : 5.957088947296143, W : tensor([0.6807, 0.6785, 0.6677]), b : 0.0079
Epoch : 40, Cost : 5.90573787689209, W : tensor([0.6826, 0.6791, 0.6652]), b : 0.0079
Epoch : 60, Cost : 5.854844093322754, W : tensor([0.6844, 0.6797, 0.6627]), b : 0.0080
Epoch : 80, Cost : 5.804478168487549, W : tensor([0.6863, 0.6803, 0.6602]), b : 0.0080
Epoch : 100, Cost : 5.754572868347168, W : tensor([0.6882, 0.6809, 0.6577]), b : 0.0080


In [11]:
# 3. nn.Module을 활용해서.
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# 데이터 준비
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  80], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

# 모델 작성
class MultilinearRegressionModel(nn.Module):
    def __init__(self):
        super(MultilinearRegressionModel, self).__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        x = self.linear(x)
        return x

# 학습
model = MultilinearRegressionModel()
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 2000
for epoch in range(nb_epochs + 1):
    prediction = model(x_train)
    cost = F.mse_loss(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch%200==0:
        print(f"Epoch {epoch} | Cost : {cost:.4f}")

Epoch 0 | Cost : 23126.0508
Epoch 200 | Cost : 2.1575
Epoch 400 | Cost : 1.9753
Epoch 600 | Cost : 1.8114
Epoch 800 | Cost : 1.6639
Epoch 1000 | Cost : 1.5309
Epoch 1200 | Cost : 1.4109
Epoch 1400 | Cost : 1.3023
Epoch 1600 | Cost : 1.2041
Epoch 1800 | Cost : 1.1150
Epoch 2000 | Cost : 1.0342


In [12]:
list(model.parameters())

[Parameter containing:
 tensor([[1.0028, 0.6679, 0.3475]], requires_grad=True),
 Parameter containing:
 tensor([0.4180], requires_grad=True)]

In [13]:
# 6. minibatch & CustomDataset
# Dataset과 DataLoader를 사용해서 학습해 보기.
# TensorDataset을 이용해서 dataset 인스턴스에 바로 담을 수도 있고, 따로 CustomDataset 클래스를 작성해도 된다.
# DataLoader사용법 : Dataset, 그리고 batch size, 그리고 shuffle 인자 (T/F). 

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, TensorDataset, DataLoader

# 데이터 준비
x_train = torch.FloatTensor([[73,  80,  75], 
                             [93,  88,  93], 
                             [89,  91,  80], 
                             [96,  98,  100],   
                             [73,  66,  70]])  
y_train = torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

# 모델 작성
class MultilinearRegressionModel(nn.Module):
    def __init__(self):
        super(MultilinearRegressionModel, self).__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        x = self.linear(x)
        return x

# Dataset 작성
dataset = TensorDataset(x_train, y_train)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# 학습하기
model = MultilinearRegressionModel()
optimizer = optim.SGD(model.parameters(), lr=1e-5)
nb_epochs = 1000
for epoch in range(nb_epochs+1):
    for batch_idx, samples in enumerate(dataloader):
        x, y = samples
        
        prediction = model(x)
        cost = F.mse_loss(prediction, y)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if epoch%100==0:
            print(f"Epoch:{epoch}, Cost:{cost:.4f}")


Epoch:0, Cost:31283.7168
Epoch:0, Cost:14801.9883
Epoch:0, Cost:4748.5518
Epoch:100, Cost:3.1086
Epoch:100, Cost:0.8268
Epoch:100, Cost:1.4172
Epoch:200, Cost:0.1347
Epoch:200, Cost:3.4244
Epoch:200, Cost:0.0498
Epoch:300, Cost:0.7679
Epoch:300, Cost:2.2122
Epoch:300, Cost:1.1518
Epoch:400, Cost:0.1054
Epoch:400, Cost:2.4241
Epoch:400, Cost:0.8306
Epoch:500, Cost:1.9649
Epoch:500, Cost:0.3883
Epoch:500, Cost:0.0244
Epoch:600, Cost:0.2439
Epoch:600, Cost:0.1579
Epoch:600, Cost:3.4027
Epoch:700, Cost:0.1865
Epoch:700, Cost:1.4582
Epoch:700, Cost:0.8745
Epoch:800, Cost:0.2825
Epoch:800, Cost:1.1191
Epoch:800, Cost:0.1020
Epoch:900, Cost:0.4407
Epoch:900, Cost:0.9676
Epoch:900, Cost:0.1438
Epoch:1000, Cost:0.1754
Epoch:1000, Cost:0.2457
Epoch:1000, Cost:1.6926


In [15]:
list(model.parameters())

[Parameter containing:
 tensor([[1.1038, 0.6552, 0.2675]], requires_grad=True),
 Parameter containing:
 tensor([0.0334], requires_grad=True)]

In [16]:
# 이번엔 CustomDataset 사용
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader

# 모델 작성
class MultilinearRegressionModel(nn.Module):
    def __init__(self):
        super(MultilinearRegressionModel, self).__init__()
        self.linear = nn.Linear(3, 1)
    
    def forward(self, x):
        x = self.linear(x)
        return x

# Dataset 작성
class CustomDataset(Dataset):
    def __init__(self):
        self.x_data  =  torch.FloatTensor([[73,  80,  75], 
                                    [93,  88,  93], 
                                    [89,  91,  80], 
                                    [96,  98,  100],   
                                    [73,  66,  70]])  
        self.y_data  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = self.x_data[idx]
        y = self.y_data[idx]
        return x, y

# 학습하기
model = MultilinearRegressionModel()
dataset = CustomDataset()
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)
optimizer = optim.SGD(model.parameters(), lr=1e-5)
nb_epochs = 2000

for epoch in range(nb_epochs+1):
    for batch_idx, samples in enumerate(dataloader):
        x, y = samples
        prediction = model(x)
        cost = F.mse_loss(prediction, y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    
        if epoch%200==0:
            print(f"Epoch : {epoch}, Cost : {cost:.4f}")

Epoch : 0, Cost : 34753.7109
Epoch : 0, Cost : 17489.5020
Epoch : 0, Cost : 2682.8364
Epoch : 200, Cost : 17.5161
Epoch : 200, Cost : 2.9873
Epoch : 200, Cost : 1.1517
Epoch : 400, Cost : 1.8183
Epoch : 400, Cost : 12.0921
Epoch : 400, Cost : 1.9551
Epoch : 600, Cost : 0.3746
Epoch : 600, Cost : 0.0328
Epoch : 600, Cost : 17.4188
Epoch : 800, Cost : 0.0275
Epoch : 800, Cost : 5.9883
Epoch : 800, Cost : 0.3663
Epoch : 1000, Cost : 0.1827
Epoch : 1000, Cost : 3.6919
Epoch : 1000, Cost : 1.6275
Epoch : 1200, Cost : 0.1884
Epoch : 1200, Cost : 2.8825
Epoch : 1200, Cost : 0.7437
Epoch : 1400, Cost : 1.2062
Epoch : 1400, Cost : 0.9640
Epoch : 1400, Cost : 0.5070
Epoch : 1600, Cost : 0.2554
Epoch : 1600, Cost : 0.1195
Epoch : 1600, Cost : 2.8323
Epoch : 1800, Cost : 0.9704
Epoch : 1800, Cost : 0.1166
Epoch : 1800, Cost : 0.2465
Epoch : 2000, Cost : 0.1550
Epoch : 2000, Cost : 0.1988
Epoch : 2000, Cost : 1.4903


In [17]:
list(model.parameters())

[Parameter containing:
 tensor([[1.1177, 0.6510, 0.2519]], requires_grad=True),
 Parameter containing:
 tensor([0.4651], requires_grad=True)]

In [18]:
# Custom Model의 define해야 하는 메소드
# 1. __init__(super init 필수), 2. forward(self, x)
# Custom Dataset의 필수 메소드
# 1. __init__(super init 필요 없음), 2. __len__, 3. __getitem__